**Notebook para iniciar o projeto Classificação de Lesões de Pele utilizando Deep Learning.**

<br>

recentes:
- Nesse link, você consegue acompanhar o que tenho feito nos últimos dias:
https://docs.google.com/document/d/17EkordHg--775S-BDvHhFxNGr9xMobtLrVhgNAioUNc/edit?usp=sharing

<br>

caminho:
* dividir dataset em treino / teste ok
* declarar modelos ok's = (VGG, )
* normalizar ok
* definir criterio de loss, optimizer, realizar treinamento
* ver acuracia com os dados de teste

<br>

anotações:
* usar AlexNet, EfficientNet, Inception, LeNet, ResNet, VGG
* usar torchvision & pytorch lightining 



In [5]:
# execute essa celula para instalar todos os requirements caso necessário.

!pip install -r requirements.txt


In [6]:

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning
import torchvision.models as models
import torchvision.transforms as transforms
from pytorch_lightning.loggers import TensorBoardLogger

from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from efficientnet_pytorch import EfficientNet

from PIL import Image
import pandas as pd
import os

# Loading CSV

In [7]:

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir_1, root_dir_2, transform):
        self.annotations = pd.read_csv(csv_file) # O caminho para o arquivo CSV que contém as informações sobre os dados.
        self.root_dir_1 = root_dir_1 # O caminho para o primeiro diretório onde estão armazenadas as imagens.
        self.root_dir_2 = root_dir_2 #O caminho para o segundo diretório onde estão armazenadas as imagens.
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_code = self.annotations.iloc[idx, 1]  # Extract the file code from the DataFrame
        img_name = img_code + '.jpg'  # Add the '.jpg' extension
        img_path = None

        # está no dir 1?
        if os.path.exists(os.path.join(self.root_dir_1, img_name)):
            img_path = os.path.join(self.root_dir_1, img_name)

        # está no dir 2?
        elif os.path.exists(os.path.join(self.root_dir_2, img_name)):
            img_path = os.path.join(self.root_dir_2, img_name)
        
        # se não estiver em nenhum dos dois diretórios, imprima uma mensagem de erro
        if img_path is None:
            print("IDX ",idx )
            print(f"File {img_name} not found in any of the specified directories.")
            return None, None  # Return None for image and label

        # se foi encontrada converte para rgb
        # essa operação é incluída por precaução para garantir que todas as imagens sejam tratadas da mesma maneira
        image = Image.open(img_path).convert('RGB')

        label = self.annotations.iloc[idx, 2] # label = acessar o valor na linha idx e coluna 2
        
        if label == 0:  # convertido em um tensor com valor 0 
            label = torch.tensor(0)
        else:
            label = torch.tensor(1) # convertido em um tensor com valor 1
        if self.transform:
            image = self.transform(image)
        return image, label


# Load CSV file and define paths
csv_file = '/home/ashiley/HAM10000_metadata_alterado.csv'
data_path_1 = '/home/ashiley/HAM10000_images_part_1'
data_path_2 = '/home/ashiley/HAM10000_images_part_2'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


# Create custom dataset instance
custom_dataset = CustomDataset(csv_file=csv_file, root_dir_1=data_path_1, root_dir_2=data_path_2, transform=transform)

# Cria um DataLoader para carregar os dados em lotes durante o treinamento.
data_loader = DataLoader(custom_dataset, batch_size=32, shuffle=True)

# Check if CUDA is available
cuda_available = torch.cuda.is_available()

if cuda_available:
    # Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("CUDA is available and {} CUDA device(s) is(are) available.".format(num_cuda_devices))
else:
    print("CUDA is not available. You are running on CPU.")

# Move a tensor to the GPU if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Concatenate datasets
full_dataset = custom_dataset  # Assuming you have already defined custom_dataset

CUDA is available and 2 CUDA device(s) is(are) available.


In [8]:
# Define the size of train and test sets
train_size = int(0.8 * len(full_dataset)) # 80% for training
test_size = len(full_dataset) - train_size # 20% for test

# Split the d|ataset into train and test sets
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create DataLoader for train and test sets
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Definicoes

In [9]:
input_channels = 3
num_classes = 2  # malignant or benign

# Models


# VGG

In [10]:
class VGGClassifier(pl.LightningModule):
    def __init__(self, num_classes):
        super(VGGClassifier, self).__init__()
        # Load the pretrained VGG16 model
        self.vgg16 = models.vgg16(pretrained=True)
        
        # Modify the first layer to accept images
        # depth (rgb = 3), number of filters 
        # self.vgg16.features[0] = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        
        # Freeze all layers except the last one
        for param in self.vgg16.parameters():
            param.requires_grad = False
        
        # Unfreeze the last layer
        for param in self.vgg16.classifier[6].parameters():
            param.requires_grad = True
            
        # Modify the classifier layer for the specified number of classes
        num_features = self.vgg16.classifier[6].in_features
        self.vgg16.classifier[6] = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.vgg16(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('val_loss', loss)
        self.log('val_acc', acc)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        return loss

vgg_model = VGGClassifier(num_classes)
print(vgg_model)

/home/ashiley/miniconda3/envs/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ashiley/miniconda3/envs/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGGClassifier(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      

In [11]:
'''# Instanciar o modelo
model = VGGClassifier(num_classes)

# Instanciar o treinador
trainer = pl.Trainer(max_epochs=10, 
                     accelerator='gpu' if cuda_available else None, 
                     logger=TensorBoardLogger("logs", name="vgg_classifier_lr1e4_freezing_all_except_last_layer_callback"),
                     callbacks=[ModelCheckpoint(monitor='val_acc', mode='max')])

# Treinar o modelo
trainer.fit(model, train_dataloader, test_dataloader)

# Testar o modelo
trainer.test(model, test_dataloader)'''

'# Instanciar o modelo\nmodel = VGGClassifier(num_classes)\n\n# Instanciar o treinador\ntrainer = pl.Trainer(max_epochs=10, \n                     accelerator=\'gpu\' if cuda_available else None, \n                     logger=TensorBoardLogger("logs", name="vgg_classifier_lr1e4_freezing_all_except_last_layer_callback"),\n                     callbacks=[ModelCheckpoint(monitor=\'val_acc\', mode=\'max\')])\n\n# Treinar o modelo\ntrainer.fit(model, train_dataloader, test_dataloader)\n\n# Testar o modelo\ntrainer.test(model, test_dataloader)'

# LeNet

In [12]:
class LeNet5(pl.LightningModule):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 6, kernel_size=5, stride=1)
        self.avg_pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.avg_pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        x = F.tanh(self.conv1(x))
        x = self.avg_pool1(x)
        x = F.tanh(self.conv2(x))
        x = self.avg_pool2(x)
        x = F.tanh(self.conv3(x))
        x = x.view(-1, 120)
        x = F.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / (len(y) * 1.0)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / (len(y) * 1.0)
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        
lenet_model = LeNet5(num_classes)
print(lenet_model)

LeNet5(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (avg_pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (avg_pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=2, bias=True)
)


In [13]:
'''# Instanciar o modelo
model = LeNet5(num_classes)

# Instanciar o treinador
trainer = pl.Trainer(max_epochs=10, 
                     accelerator='gpu' if cuda_available else None, 
                     logger=TensorBoardLogger("logs", name="lenet5_lr1e3_freezing_all_except_last_layer_callback"),
                     callbacks=[ModelCheckpoint(monitor='val_acc', mode='max')])

# Treinar o modelo
trainer.fit(model, train_dataloader, test_dataloader)

# Testar o modelo
trainer.test(model, test_dataloader)
'''

'# Instanciar o modelo\nmodel = LeNet5(num_classes)\n\n# Instanciar o treinador\ntrainer = pl.Trainer(max_epochs=10, \n                     accelerator=\'gpu\' if cuda_available else None, \n                     logger=TensorBoardLogger("logs", name="lenet5_lr1e3_freezing_all_except_last_layer_callback"),\n                     callbacks=[ModelCheckpoint(monitor=\'val_acc\', mode=\'max\')])\n\n# Treinar o modelo\ntrainer.fit(model, train_dataloader, test_dataloader)\n\n# Testar o modelo\ntrainer.test(model, test_dataloader)\n'

# ResNet

In [14]:
class ResNetClassifier(pl.LightningModule):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        # Load a pre-trained ResNet model
        self.resnet = models.resnet18(pretrained=True)
        
        # Freeze all layers except the last one
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        # Unfreeze the last layer
        for param in self.resnet.fc.parameters(): 
            param.requires_grad = True
        
        # Modify the last fully connected layer to match the number of classes
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.resnet(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        
resnet_model = models.resnet18(num_classes=num_classes)
print(resnet_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
'''# Instanciar o modelo
model = ResNetClassifier(num_classes)

# Instanciar o treinador
trainer = pl.Trainer(max_epochs=10, 
                     accelerator='gpu' if cuda_available else None, 
                     logger=TensorBoardLogger("logs", name="resnet_lr1e3_freezing_all_except_last_layer"),
                     callbacks=[ModelCheckpoint(monitor='val_acc', mode='max')])

# Treinar o modelo
trainer.fit(model, train_dataloader, test_dataloader)

# Testar o modelo
trainer.test(model, test_dataloader)'''


/home/ashiley/miniconda3/envs/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: logs/resnet_lr1e3_freezing_all_except_last_layer
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name   | Type   | Para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/ashiley/miniconda3/envs/env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


/home/ashiley/miniconda3/envs/env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 124/124 [01:09<00:00,  1.79it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 124/124 [01:09<00:00,  1.79it/s, v_num=0]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/ashiley/miniconda3/envs/env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.



Testing DataLoader 0: 100%|██████████| 31/31 [00:14<00:00,  2.15it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8577215075492859
        test_loss           0.32642561197280884
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.32642561197280884, 'test_acc': 0.8577215075492859}]

# AlexNet

In [34]:
class AlexNetClassifier(pl.LightningModule):
    def __init__(self, num_classes):
        super(AlexNetClassifier, self).__init__()
        self.alexnet = models.alexnet(pretrained=True)
        
        # Freeze all layers except the last one
        for param in self.alexnet.parameters():
            param.requires_grad = False
        
        # Unfreeze the last layer
        for param in self.alexnet.classifier[6].parameters():
            param.requires_grad = True
            
        # Replace the classifier layer for the specified number of classes
        num_features = self.alexnet.classifier[6].in_features
        self.alexnet.classifier[6] = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.alexnet(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        
alexnet_model = models.alexnet(num_classes=num_classes)
print(alexnet_model)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [35]:
'''# Instanciar o modelo
model = AlexNetClassifier(num_classes)

# Instanciar o treinador
trainer = pl.Trainer(max_epochs=10, 
                     accelerator='gpu' if cuda_available else None, 
                     logger=TensorBoardLogger("logs", name="alexnet_lr1e3_freezing_all_except_last_layer"),
                     callbacks=[ModelCheckpoint(monitor='val_acc', mode='max')])

# Treinar o modelo
trainer.fit(model, train_dataloader, test_dataloader)

# Testar o modelo
trainer.test(model, test_dataloader)'''

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: logs/alexnet_lr1e3_freezing_all_except_last_layer
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type    | Params
------------------------------------
0 | alexnet | AlexNet | 57.0 M
------------------------------------
8.2 K     Trainable params
57.0 M    Non-trainable params
57.0 M    Total params
228.048   Total estimated model params size (MB)


Epoch 9: 100%|██████████| 124/124 [01:12<00:00,  1.71it/s, v_num=0]        

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 124/124 [01:12<00:00,  1.71it/s, v_num=0]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



Testing DataLoader 0: 100%|██████████| 31/31 [00:13<00:00,  2.24it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8379746675491333
        test_loss           0.3522632122039795
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.3522632122039795, 'test_acc': 0.8379746675491333}]

# EfficientNet

In [36]:
class EfficientNetClassifier(pl.LightningModule):
    def __init__(self, num_classes):
        super(EfficientNetClassifier, self).__init__()

        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0', num_classes=num_classes)
        
    def forward(self, x):
        return self.efficientnet(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        
efficientnet_model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=num_classes)       
print(efficientnet_model)

Loaded pretrained weights for efficientnet-b0
EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kern

In [ ]:
'''# Instanciar o modelo
model = EfficientNetClassifier(num_classes)

# Instanciar o treinador
trainer = pl.Trainer(max_epochs=10, 
                     accelerator='gpu' if cuda_available else None, 
                     logger=TensorBoardLogger("logs", name="efficientnetclassifier_lr1e3_freezing_all_except_last_layer"),
                     callbacks=[ModelCheckpoint(monitor='val_acc', mode='max')])

# Treinar o modelo
trainer.fit(model, train_dataloader, test_dataloader)

# Testar o modelo
trainer.test(model, test_dataloader)'''

# Inception

In [18]:
class InceptionClassifier(pl.LightningModule):
    def __init__(self, num_classes=2):
        super().__init__()
        self.inception = models.inception_v3(pretrained=True)
        
        # Replace the classifier layer for the specified number of classes
        in_features = self.inception.fc.in_features
        self.inception.fc = nn.Linear(in_features, num_classes)
        
    def forward(self, x):
        return self.inception(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        
# Instantiate the model
inception_model = InceptionClassifier(num_classes=2)
print(inception_model)


/home/ashiley/miniconda3/envs/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


InceptionClassifier(
  (inception): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a